Following tutorial described here: https://docs.prefect.io/tutorials/first-steps/

In [2]:
from prefect import flow

In [3]:
@flow
def my_favorite_function():
    print("What is your favorite number?")
    return 42

In [4]:
print(my_favorite_function())

02:40:41.707 | INFO    | prefect.engine - Created flow run 'quantum-buffalo' for flow 'my-favorite-function'
02:40:42.124 | INFO    | Flow run 'quantum-buffalo' - Finished in state Completed()


What is your favorite number?
42


### Tasks are called from flows

In [6]:
import requests
from prefect import flow, task

@task(name='task_call_api', description='Calls API')
def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

@task(name='task_parse_response', description='Parses response')
def parse_fact(response):
    fact = response["fact"]
    print(fact)
    return fact

@flow(name='flow_data_extraction', description='Calls API and parses response')
def api_flow(url):
    fact_json = call_api(url)
    fact_text = parse_fact(fact_json)
    return fact_text

api_flow("https://catfact.ninja/fact")

00:13:02.084 | INFO    | prefect.engine - Created flow run 'impartial-pillbug' for flow 'flow_data_extraction'
00:13:02.593 | INFO    | Flow run 'impartial-pillbug' - Created task run 'task_call_api-ded10bed-0' for task 'task_call_api'
00:13:02.593 | INFO    | Flow run 'impartial-pillbug' - Executing 'task_call_api-ded10bed-0' immediately...
00:13:03.590 | INFO    | Task run 'task_call_api-ded10bed-0' - Finished in state Completed()
00:13:03.648 | INFO    | Flow run 'impartial-pillbug' - Created task run 'task_parse_response-6803447a-0' for task 'task_parse_response'
00:13:03.656 | INFO    | Flow run 'impartial-pillbug' - Executing 'task_parse_response-6803447a-0' immediately...


200


00:13:03.832 | INFO    | Task run 'task_parse_response-6803447a-0' - Finished in state Completed()
00:13:03.914 | INFO    | Flow run 'impartial-pillbug' - Finished in state Completed()


The biggest wildcat today is the Siberian Tiger. It can be more than 12 feet (3.6 m) long (about the size of a small car) and weigh up to 700 pounds (317 kg).


'The biggest wildcat today is the Siberian Tiger. It can be more than 12 feet (3.6 m) long (about the size of a small car) and weigh up to 700 pounds (317 kg).'

### Tasks and flows both can be retried
#### https://docs.prefect.io/tutorials/flow-task-config/#flow-and-task-retries

In [11]:
from prefect import flow, task

# this tasks runs 3 times before the flow fails
@task(retries=1, retry_delay_seconds=10)
def failure():
    print('running')
    raise ValueError("bad code")

@flow
def test_retries():
    return failure()

#test_retries()

c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\prefect\tasks.py:167: UserWarning: A task named 'failure' and defined at 'C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3012\3272433379.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\prefect\flows.py:176: UserWarning: A flow named 'test-retries' and defined at 'C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3012\3272433379.py:9' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


### Task runners
#### https://docs.prefect.io/tutorials/execution/

Different Types of task runners are:

ConcurrentTaskRunner: Runs tasks concurrently.
SequentialTaskRunner: Runs tasks sequentially.
DaskTaskRunner: Distributed / Parallel computing. 

ConcurrentTaskRunner

In [12]:
import time
from prefect import task, flow

@task
def print_values(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow
def my_flow():
    print_values.submit(["AAAA"] * 15)
    print_values.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow()

01:02:45.071 | INFO    | prefect.engine - Created flow run 'delightful-sparrow' for flow 'my-flow'
01:02:45.437 | INFO    | Flow run 'delightful-sparrow' - Created task run 'print_values-0bb9a2c3-1' for task 'print_values'
01:02:45.437 | INFO    | Flow run 'delightful-sparrow' - Submitted task run 'print_values-0bb9a2c3-1' for execution.
01:02:45.477 | INFO    | Flow run 'delightful-sparrow' - Created task run 'print_values-0bb9a2c3-0' for task 'print_values'
01:02:45.477 | INFO    | Flow run 'delightful-sparrow' - Submitted task run 'print_values-0bb9a2c3-0' for execution.


01:02:50.790 | INFO    | Task run 'print_values-0bb9a2c3-1' - Finished in state Completed()


01:02:53.388 | INFO    | Task run 'print_values-0bb9a2c3-0' - Finished in state Completed()
01:02:53.477 | INFO    | Flow run 'delightful-sparrow' - Finished in state Completed('All states completed.')


SequentialTaskRunner

In [13]:
import time
from prefect import task, flow
from prefect.task_runners import SequentialTaskRunner

@task
def print_values(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow(task_runner=SequentialTaskRunner())
def my_flow():
    print_values.submit(["AAAA"] * 15)
    print_values.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow()

c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\prefect\tasks.py:167: UserWarning: A task named 'print_values' and defined at 'C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3012\1773945071.py:5' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\prefect\flows.py:176: UserWarning: A flow named 'my-flow' and defined at 'C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3012\1773945071.py:11' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(
01:04:56.362 | INFO    | prefect.engine - Created flow run 'meticulous-dragon' for flow 'my-flow'
01:04:56.766 | INFO    | Flow run 'meticulous-dragon' - Created task run 'print_values-0bb

01:05:04.605 | INFO    | Task run 'print_values-0bb9a2c3-0' - Finished in state Completed()
01:05:04.687 | INFO    | Flow run 'meticulous-dragon' - Created task run 'print_values-0bb9a2c3-1' for task 'print_values'
01:05:04.687 | INFO    | Flow run 'meticulous-dragon' - Executing 'print_values-0bb9a2c3-1' immediately...


01:05:09.994 | INFO    | Task run 'print_values-0bb9a2c3-1' - Finished in state Completed()


01:05:10.098 | INFO    | Flow run 'meticulous-dragon' - Finished in state Completed('All states completed.')


### To checkout Prefect UI, run command: prefect orion start

https://docs.prefect.io/tutorials/orion/

Deploy Prefect workflows:
https://docs.prefect.io/tutorials/deployments/